# Application of Foursquare Data to an Educational Consultancy Business

In [54]:
# Importing libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install --yes BeautifulSoup4
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [55]:
## Part 1: Web Scraping

In [56]:
# Use the requests to download the data from Wikipedia
website_data = requests.get('https://www.doogal.co.uk/london_stations.php')

# Use BeautifulSoup to parse the table, inspect source to get the relevant class
soup = BeautifulSoup(website_data.text, 'html.parser')
table = soup.find('table', attrs={'class':'sortable stationsTable table table-striped table-hover'})

# 'table' is a table in HTML, let's parse this and turn this into a dataframe
rows = table.findAll('tr')
station_list = []
zone_list = []
latitude_list = []
longitude_list = []

for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    station_list.append(cols[0])
    zone_list.append(cols[1])
    latitude_list.append(float(cols[3]))
    longitude_list.append(float(cols[4]))

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df = pd.DataFrame({
    'Neighborhood':station_list, 
    'zone':zone_list, 
    'latitude':latitude_list,
    'longitude':longitude_list
    })

print("\nLet's have a look at the first 10 records\n")
print(df.head(10))

print("\nLet's have a look at the shape of the data frame\n")
print(df.shape)

print("\nLet's have a look at some of the meta data\n")
df.info()

count_before = len(df)
df = df[df['zone'].str.contains("2")]
count_after = len(df)
print("\nThe student is only interested in living in zone 2, records before: {}, records after: {}\n".format(count_before, count_after))
print(df.head(10))


Let's have a look at the first 10 records

        Neighborhood     zone   latitude  longitude
0         Abbey Road        3  51.531952   0.003723
1         Abbey Wood        4  51.490784   0.120272
2      Acton Central        2  51.508758  -0.263430
3    Acton Main Line        3  51.516887  -0.267690
4         Acton Town        3  51.503071  -0.280303
5  Addington Village  3,4,5,6  51.356239  -0.032665
6         Addiscombe  3,4,5,6  51.379808  -0.073213
7        Albany Park        5  51.435816   0.126445
8            Aldgate        1  51.514342  -0.075627
9       Aldgate East        1  51.515082  -0.073001

Let's have a look at the shape of the data frame

(653, 4)

Let's have a look at some of the meta data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653 entries, 0 to 652
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Neighborhood  653 non-null    object 
 1   zone          653 non-null    object 
 2 

## Part 2: Explore and cluster the neighborhoods in London, Zone 2

Let's get the geographical coordinates of London, UK.

In [57]:
address = 'London, United Kingdom'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of London, United Kingdom are 51.5073219, -0.1276474.


Let's visualize the neighborhoods of London.

In [62]:
# create map of Manhattan using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df['latitude'], df['longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [63]:
CLIENT_ID = 'NDIQLTKPFI1AH3YHT3AZM0O01D4FR1FMZBAMPSYC5WVAWSY1' # your Foursquare ID
CLIENT_SECRET = '52VXNKYITCXBJ05KFBY5ZBQQIHB1VXHOOWHIB03GCO1YVR3W' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NDIQLTKPFI1AH3YHT3AZM0O01D4FR1FMZBAMPSYC5WVAWSY1
CLIENT_SECRET:52VXNKYITCXBJ05KFBY5ZBQQIHB1VXHOOWHIB03GCO1YVR3W


In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called _london_venues_.

In [66]:
london_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Acton Central
All Saints
Archway
Arsenal
Barons Court
Battersea Park
Belsize Park
Bermondsey
Bethnal Green
Bethnal Green Rail
Blackwall
Bow Church
Bow Road
Brixton
Brockley
Bromley-by-Bow
Brondesbury
Brondesbury Park
Caledonian Road
Caledonian Road and Barnsbury
Cambridge Heath
Camden Road
Camden Town
Canada Water
Canary Wharf
Canonbury
Chalk Farm
Clapham Common
Clapham High Street
Clapham Junction
Clapham North
Clapham South
Clapton
Crossharbour and London Arena
Cutty Sark for Maritime Greenwich
Dalston Junction
Dalston Kingsland
Denmark Hill
Deptford
Deptford Bridge
Devons Road
Drayton Park
Earls Court
East Acton
East Dulwich
East India
East Putney
Elephant and Castle
Elverson Road
Essex Road
Finchley Road
Finchley Road and Frognal
Finsbury Park
Fulham Broadway
Goldhawk Road
Gospel Oak
Greenwich
Hackney Central
Hackney Downs
Hackney Wick
Haggerston
Hammersmith (District)
Hammersmith (Met.)
Hampstead
Herne Hill
Heron Quays
Highbury and Islington
Holland Park
Holloway Road
Homerton
Hox

Let's check the size of the resulting dataframe

In [67]:
print(london_venues.shape)
london_venues.head()

(5938, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton Central,51.508758,-0.26343,The Station House,51.508877,-0.263076,Pub
1,Acton Central,51.508758,-0.26343,The Rocket,51.508772,-0.263787,Pub
2,Acton Central,51.508758,-0.26343,Acton Park,51.508595,-0.261573,Park
3,Acton Central,51.508758,-0.26343,Acton Centre,51.506608,-0.266878,Gym / Fitness Center
4,Acton Central,51.508758,-0.26343,MrBakeme,51.508452,-0.268543,Creperie


Let's check how many venues were returned for each neighborhood

In [68]:
london_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acton Central,19,19,19,19,19,19
All Saints,11,11,11,11,11,11
Archway,33,33,33,33,33,33
Arsenal,32,32,32,32,32,32
Barons Court,38,38,38,38,38,38
Battersea Park,20,20,20,20,20,20
Belsize Park,40,40,40,40,40,40
Bermondsey,28,28,28,28,28,28
Bethnal Green,54,54,54,54,54,54


Let's find out how many unique categories can be curated from all the returned venues

In [69]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 327 uniques categories.


In [70]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighborhood,ATM,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comedy Club,Community Center,Community College,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South 

And let's examine the new dataframe size.

In [71]:
london_onehot.shape

(5938, 328)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [72]:
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped

,Neighborhood,ATM,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comedy Club,Community Center,Community College,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South 

Let's confirm the new size

In [73]:
london_grouped.shape

(148, 328)

Let's print each neighborhood along with the top 5 most common venues

In [74]:
num_top_venues = 5

for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acton Central----
                  venue  freq
0                   Pub  0.21
1  Gym / Fitness Center  0.21
2           Coffee Shop  0.11
3         Train Station  0.11
4                  Park  0.05


----All Saints----
                venue  freq
0       Grocery Store  0.09
1  Chinese Restaurant  0.09
2          Steakhouse  0.09
3                 Bar  0.09
4                Park  0.09


----Archway----
                venue  freq
0         Coffee Shop  0.18
1       Grocery Store  0.15
2                 Pub  0.12
3         Pizza Place  0.09
4  Italian Restaurant  0.06


----Arsenal----
                  venue  freq
0                   Pub  0.09
1                  Café  0.09
2  Gym / Fitness Center  0.09
3           Pizza Place  0.09
4        Soccer Stadium  0.09


----Barons Court----
           venue  freq
0            Pub  0.16
1    Coffee Shop  0.13
2           Café  0.13
3          Hotel  0.08
4  Grocery Store  0.08


----Battersea Park----
                venue  freq
0          

Let's put that into a _pandas_ dataframe. First, let's write a function to sort the venues in descending order.

In [75]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton Central,Gym / Fitness Center,Pub,Coffee Shop,Train Station,Bakery,Mini Golf,Park,Supermarket,Wine Shop,Creperie
1,All Saints,Bar,Café,Park,Sandwich Place,English Restaurant,Outdoor Sculpture,Coffee Shop,Grocery Store,Chinese Restaurant,Fish Market
2,Archway,Coffee Shop,Grocery Store,Pub,Pizza Place,Italian Restaurant,Asian Restaurant,Café,Gastropub,Farmers Market,Sandwich Place
3,Arsenal,Café,Pizza Place,Pub,Gym / Fitness Center,Soccer Stadium,Sports Bar,Park,Train Station,Mediterranean Restaurant,Burger Joint
4,Barons Court,Pub,Café,Coffee Shop,Hotel,Grocery Store,Thai Restaurant,Indian Restaurant,Performing Arts Venue,Garden,Chinese Restaurant


Run _k_-means to cluster the neighborhood into 5 clusters.

In [77]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 3, 3, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [78]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = df

# merge london_merged with london_merged to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

london_merged.head() # check the last columns!

,Neighborhood,zone,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Acton Central,2,51.508758,-0.263430,4,Gym / Fitness Center,Pub,Coffee Shop,Train Station,Bakery,Mini Golf,Park,Supermarket,Wine Shop,Creperie
11,All Saints,2,51.510477,-0.012625,3,Bar,Café,Park,Sandwich Place,English Restaurant,Outdoor Sculpture,Coffee Shop,Grocery Store,Chinese Restaurant,Fish Market
18,Archway,"2,3",51.565491,-0.135122,3,Coffee Shop,Grocery Store,Pub,Pizza Place,Italian Restaurant,Asian Restaurant,Café,Gastropub,Farmers Market,Sandwich Place
21,Arsenal,2,51.558541,-0.105500,0,Café,Pizza Place,Pub,Gym / Fitness Center,Soccer Stadium,Sports Bar,Park,Train Station,Mediterranean Restaurant,Burger Joint
33,Barons Court,2,51.490229,-0.213430,0,Pub,Café,Coffee Shop,Hotel,Grocery Store,Thai Restaurant,Indian Restaurant,Performing Arts Venue,Garden,Chinese Restaurant


Finally, let's visualize the resulting clusters

In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['latitude'], london_merged['longitude'], london_merged['Neighborhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1 appears to describe areas which contain a lot of restaurants

In [82]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,2,Café,Pizza Place,Pub,Gym / Fitness Center,Soccer Stadium,Sports Bar,Park,Train Station,Mediterranean Restaurant,Burger Joint
33,2,Pub,Café,Coffee Shop,Hotel,Grocery Store,Thai Restaurant,Indian Restaurant,Performing Arts Venue,Garden,Chinese Restaurant
47,2,Café,Pub,Bakery,Deli / Bodega,Pizza Place,Hotel,Coffee Shop,Bookstore,Thai Restaurant,Grocery Store
49,2,Pub,Coffee Shop,Grocery Store,Café,Park,Farmers Market,Brazilian Restaurant,Gym / Fitness Center,Beer Bar,Gelato Shop
51,2,Pub,Coffee Shop,Café,Cocktail Bar,Park,Art Gallery,Beer Bar,Gym,Grocery Store,Hotel
52,2,Pub,Café,Coffee Shop,Grocery Store,Pizza Place,Fast Food Restaurant,Park,Turkish Restaurant,Bakery,Gym
77,"2,3",Café,Historic Site,Supermarket,Metro Station,Pub,Waterfront,Bus Stop,Light Rail Station,Bike Rental / Bike Share,Canal Lock
78,2,Coffee Shop,Café,Pub,Pizza Place,Park,Portuguese Restaurant,Theater,Deli / Bodega,Convenience Store,Doner Restaurant
79,2,Park,Pub,Gym / Fitness Center,Coffee Shop,Farmers Market,Café,Japanese Restaurant,Flower Shop,Flea Market,Fishing Spot
109,2,Café,Bar,Italian Restaurant,Pub,Coffee Shop,Bakery,Pizza Place,Restaurant,Music Venue,French Restaurant


In [87]:
[1] + list(range(5, london_merged.shape[1]

SyntaxError: unexpected EOF while parsing (<ipython-input-87-6aa9e3ac6390>, line 1)

Cluster 2 - Isolated cluster with garden as the only element

In [83]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,2,Bar,Pub,Coffee Shop,Sports Bar,Furniture / Home Store,Café,Gastropub,Chinese Restaurant,Grocery Store,Track Stadium
66,2,Pub,Convenience Store,Bus Stop,Metro Station,Grocery Store,Gym,Park,Burger Joint,Coffee Shop,Bar
67,2,Pub,Bus Stop,Park,Metro Station,Chinese Restaurant,Locksmith,Burger Joint,Hotel,Bar,Coffee Shop
96,2,Pub,Bus Stop,Café,Coffee Shop,Park,Thai Restaurant,Platform,Organic Grocery,Fish & Chips Shop,Modern European Restaurant
133,2,Pub,Grocery Store,Chinese Restaurant,Pizza Place,Gym / Fitness Center,Park,Train Station,Turkish Restaurant,Breakfast Spot,Café
161,"2,3",Pub,Gym / Fitness Center,Grocery Store,Hotel,Japanese Restaurant,River,Light Rail Station,Park,Athletics & Sports,Vietnamese Restaurant
162,2,Pub,Light Rail Station,Fish & Chips Shop,Gym,Fried Chicken Joint,Dance Studio,Event Space,Exhibit,Falafel Restaurant,Farm
171,2,Chinese Restaurant,Pub,Video Game Store,Furniture / Home Store,Fish & Chips Shop,Metro Station,Falafel Restaurant,Grocery Store,Bakery,Pool Hall
173,2,Pub,Pizza Place,Coffee Shop,Park,ATM,Turkish Restaurant,English Restaurant,Grocery Store,Supermarket,Burger Joint
184,"1,2",Pub,Chinese Restaurant,Pizza Place,Gym,Park,Coffee Shop,Cantonese Restaurant,Bike Rental / Bike Share,Street Food Gathering,Market


Cluster 3 - Small cluster with parks and gyms

In [84]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
303,2,Pub,Train Station,Bakery,Portuguese Restaurant,Indian Chinese Restaurant,Film Studio,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant


Cluster 4 - Isolated cluster with swim school as the only element

In [85]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,2,Bar,Café,Park,Sandwich Place,English Restaurant,Outdoor Sculpture,Coffee Shop,Grocery Store,Chinese Restaurant,Fish Market
18,"2,3",Coffee Shop,Grocery Store,Pub,Pizza Place,Italian Restaurant,Asian Restaurant,Café,Gastropub,Farmers Market,Sandwich Place
61,2,Hotel,Sandwich Place,Italian Restaurant,Coffee Shop,Outdoor Sculpture,Gym / Fitness Center,Chinese Restaurant,Bar,Steakhouse,Café
73,2,Caribbean Restaurant,Market,Pizza Place,Coffee Shop,Pub,Cocktail Bar,Beer Bar,Fried Chicken Joint,Sandwich Place,Italian Restaurant
74,2,Coffee Shop,Gastropub,Beer Store,Grocery Store,Indian Restaurant,Train Station,Deli / Bodega,Pizza Place,Pub,Convenience Store
87,2,Café,Grocery Store,Rental Car Location,Yoga Studio,Park,Coffee Shop,Theater,Breakfast Spot,Restaurant,Convenience Store
88,2,Park,Coffee Shop,Grocery Store,Brewery,Rental Car Location,Fast Food Restaurant,Greek Restaurant,Bar,Gastropub,Caucasian Restaurant
89,2,Coffee Shop,Pub,Hotel,Cocktail Bar,Flower Shop,Music Venue,Breakfast Spot,Park,Canal,Pizza Place
90,2,Pub,Coffee Shop,Café,Market,Burger Joint,Italian Restaurant,Ice Cream Shop,Beer Bar,Greek Restaurant,Music Venue
91,2,Coffee Shop,Pub,Greek Restaurant,Italian Restaurant,Pizza Place,Bar,Market,Tea Room,Beer Bar,Supermarket


Cluster 5 - Isolated cluster with trail as the only element

In [86]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,2,Gym / Fitness Center,Pub,Coffee Shop,Train Station,Bakery,Mini Golf,Park,Supermarket,Wine Shop,Creperie
331,2,Gym / Fitness Center,Indoor Play Area,Latin American Restaurant,Coffee Shop,Park,Food & Drink Shop,Farm,Event Space,French Restaurant,Exhibit
590,2,Gym / Fitness Center,Pub,Deli / Bodega,Grocery Store,Restaurant,Brewery,Dance Studio,Cupcake Shop,Exhibit,Falafel Restaurant
